In [3]:
import os
from skimage import io, filters, transform

def threshold_images(input_folder, output_folder, max_width, max_height, threshold_value):

    # Create output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Loop through each file in the input folder
    for filename in os.listdir(input_folder):
        if filename.endswith('.png'):
            # Read the image
            img_path = os.path.join(input_folder, filename)
            image = io.imread(img_path, as_gray=True)

            # Resize the image
            # Calculate the scaling factor to fit within the specified range
            scale_factor = min(max_width / image.shape[1], max_height / image.shape[0])
            resized_image = transform.resize(image, (int(image.shape[0] * scale_factor), int(image.shape[1] * scale_factor)))

            # Sharpening (using unsharp masking)
            sharpened_image = filters.unsharp_mask(resized_image, radius=3, amount=3)
            
            # Apply thresholding
            thresholded_image = sharpened_image > threshold_value

            # Sharpening (using unsharp masking)
            sharpened_image = filters.unsharp_mask(thresholded_image, radius=1, amount=1)
            
            # Save thresholded image to output folder
            output_path = os.path.join(output_folder, filename)
            io.imsave(output_path, sharpened_image.astype('uint8') * 255)  # Convert boolean array to uint8


#input test
# input_folder = 'C:/Users/jieru/Downloads/FIT3164/project/Medication-Dispensing-with-Image-Processing/image processing/x'
# output_folder = "C:/Users/jieru/Downloads/FIT3164/project/Medication-Dispensing-with-Image-Processing/image processing/enhanced_images"

input_folder = 'D:/2024/FYP/DATASETS/mydispense_meds/mydispense_meds/testing images'
output_folder = 'D:/2024/FYP/DATASETS/mydispense_meds/processed_images'

#actual folder
# input_folder = 'C:/Users/jieru/Downloads/FIT3164/project/Medication-Dispensing-with-Image-Processing/image processing/mydispense_meds_proceessing'
# output_folder = 'C:/Users/jieru/Downloads/FIT3164/project/Medication-Dispensing-with-Image-Processing/image processing/resized_images'
max_width = 600  # Specify your maximum width
max_height = 600  # Specify your maximum height
threshold_value=0.76 #0.76

# Apply try_all_threshold to PNG images in the input folder
threshold_images(input_folder, output_folder, max_width, max_height, threshold_value)

In [ ]:
%pip install tensorflow

In [ ]:
""" i really hate this """
import tensorflow as tf 
from tensorflow.keras import layers, models

# define input shape, required?

def backbone(input_shape):
    # we use a bunch of CNN, which may need to be refined later
    input_img = layers.Input(shape=input_shape)
    x = layers.Conv2D(32, (3, 3), padding='same', activation='relu')(input_img)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Conv2D(64, (3, 3), padding='same', activation='relu')(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Conv2D(128, (3, 3), padding='same', activation='relu')(x)
    x = layers.MaxPooling2D((2, 2))(x)
    # Additional layers can be added as needed
    return models.Model(input_img, x)

def feature_giant(bbmodel):
    input_features = bbmodel.output
    # upsample
    x = layers.Conv2DTranspose(128, (3, 3), strides=(2, 2), padding='same', activation='relu')(input_features)
    x = layers.Conv2DTranspose(64, (3, 3), strides=(2, 2), padding='same', activation='relu')(x)
    x = layers.Conv2DTranspose(32, (3, 3), strides=(2, 2), padding='same', activation='relu')(x)
    return models.Model(bbmodel.input, x)

# prediction maps
def prediction_maps(feature_giant_model):
    input_img = feature_giant_model.input
    features = feature_giant_model.output

    # Predicted probability map (binary classification)
    prob_map = layers.Conv2D(1, (1, 1), activation='sigmoid', name='prob_map')(features)
    
    # Predicted threshold map
    threshold_map = layers.Conv2D(1, (1, 1), activation='sigmoid', name='threshold_map')(features)
    
    return models.Model(input_img, [prob_map, threshold_map])

def approx_binarization(prob, threshold):
    binarized_map = tf.keras.backend.greater_equal(prob_map, threshold_map)
    return binarized_map

def binmodel(shape):
    # Create the backbone and feature giant models
    backbone_model = backbone(input_shape)
    feature_giant_model = feature_giant(backbone_model)
    
    # Create the prediction maps model
    prediction_maps_model = prediction_maps(feature_giant_model)
    
    # Define the input and outputs
    input_img = prediction_maps_model.input
    prob_map, threshold_map = prediction_maps_model.output
    
    # Calculate the binarized map
    binarized_map = approx_binarization(prob_map, threshold_map)
    
    # Create the final model
    return models.Model(input_img, [prob_map, threshold_map, binarized_map])

input_shape = (128, 128, 3)  # Adjust input shape as needed
model = binmodel(input_shape)

# Compile the model
model.compile(optimizer='adam', loss={'prob_map': 'binary_crossentropy',
                                      'threshold_map': 'mse',
                                      'binarized_map': 'binary_crossentropy'})



In [ ]:
def bounding():
    ''' dbnet algorithm used, with this hopefully we can by pass the binarization problem '''
    pass

def meat():
    pass